In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:

!pip install scikit-surprise --no-binary :all:

In [3]:
!wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

--2025-07-04 04:02:48--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  3.34MB/s    in 0.3s    

2025-07-04 04:02:49 (3.34 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [4]:
#📘 Movie Recommender with NumPy SVD 🍿🎦🎥🎞️👇




import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error




# 🔹 Step 1: Load Ratings and Movies
ratings = pd.read_csv("ml-latest-small/ratings.csv")
movies = pd.read_csv("ml-latest-small/movies.csv")




# 🔹 Step 2: Create User-Item Matrix
user_item_matrix = ratings.pivot(index='userId', columns='movieId', values='rating')
user_item_matrix.fillna(0, inplace=True)  # Fill missing ratings with 0




# 🔹 Step 3: Apply SVD
U, sigma, Vt = np.linalg.svd(user_item_matrix, full_matrices=False)
sigma = np.diag(sigma)




# 🔹 Step 4: Reconstruct Ratings
k = 20  # number of latent features
reconstructed = np.dot(np.dot(U[:, :k], sigma[:k, :k]), Vt[:k, :])






# 🔹 Step 5: Recommend Top N Movies for a User
def recommend_movies(user_id, original_matrix, predicted_matrix, movie_df, n=5):
    user_index = user_id - 1  # because userId starts at 1
    predicted_ratings = predicted_matrix[user_index]
    user_seen = original_matrix.iloc[user_index] > 0
    predicted_ratings[user_seen.values] = -1  # exclude seen movies

    top_n_indices = np.argsort(predicted_ratings)[::-1][:n]
    movie_ids = original_matrix.columns[top_n_indices]

    recommendations = movie_df[movie_df['movieId'].isin(movie_ids)]
    print(f"\n🎬 Top {n} Recommendations for User {user_id}:")
    for mid in movie_ids:
        title = movie_df.loc[movie_df.movieId == mid, 'title'].values[0]
        score = predicted_ratings[original_matrix.columns.get_loc(mid)]
        print(f"{title} (Predicted Rating: {score:.2f})")



# 🔹 Step 6: Evaluate
mask = user_item_matrix.values > 0
rmse = np.sqrt(mean_squared_error(user_item_matrix.values[mask], reconstructed[mask]))
mae = mean_absolute_error(user_item_matrix.values[mask], reconstructed[mask])

print(f"\n✅ RMSE: {rmse:.4f}")
print(f"✅ MAE: {mae:.4f}")

# 🔹 Step 7: Show Recommendations for Few Users
recommend_movies(1, user_item_matrix, reconstructed, movies)
recommend_movies(2, user_item_matrix, reconstructed, movies)
recommend_movies(3, user_item_matrix, reconstructed, movies)


✅ RMSE: 2.3662
✅ MAE: 1.9820

🎬 Top 5 Recommendations for User 1:
Terminator 2: Judgment Day (1991) (Predicted Rating: 3.93)
Aliens (1986) (Predicted Rating: 3.73)
Sixth Sense, The (1999) (Predicted Rating: 3.33)
Breakfast Club, The (1985) (Predicted Rating: 3.32)
Die Hard (1988) (Predicted Rating: 3.16)

🎬 Top 5 Recommendations for User 2:
Fight Club (1999) (Predicted Rating: 1.29)
Pulp Fiction (1994) (Predicted Rating: 0.95)
Forrest Gump (1994) (Predicted Rating: 0.94)
American History X (1998) (Predicted Rating: 0.88)
Matrix, The (1999) (Predicted Rating: 0.84)

🎬 Top 5 Recommendations for User 3:
Aliens (1986) (Predicted Rating: 0.23)
Alien (1979) (Predicted Rating: 0.21)
Lethal Weapon (1987) (Predicted Rating: 0.20)
Saving Private Ryan (1998) (Predicted Rating: 0.19)
RoboCop (1987) (Predicted Rating: 0.19)
